# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
print("Libraries imported")

Libraries imported


#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

In [3]:
print(f"Total files found: {len(file_path_list)}")

Total files found: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
             
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

print("New CSV file generated")

8056
New CSV file generated


In [5]:
# number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection established successfully")
except Exception as e:
    print(f"Oops: {e}")


Connection established successfully


#### Create Keyspace

In [7]:
try: 
    session.execute(
        """
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
        """
    )
    print("Keyspace created successfully")
except Exception as e:
    print(f"Oops: {e}")
    

Keyspace created successfully


#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
    print("Session connected")
except Exception as e:
    print(f"Oops: {e}")

Session connected


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Table 1 Description: The table has artist, song, length, sessionId and itemInSession columns. Primary key is (sessionId, itemInSession) where sessionId represents 
# partition key and itemInSession represnts clustering column as we need to run query1 for particular values of sessionId and itemInSession. 

try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS session_item_table
        (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))
        """
    )
    print("Table 1 created successfully")
except Exception as e:
    print(f"Oops: {e}")


Table 1 created successfully


In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item_table (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        
print("Table 1 filled successfully")

Table 1 filled successfully


#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
try: 
    rows = session.execute(
        """
        SELECT artist, song, length FROM session_item_table WHERE sessionId=338 and itemInSession = 4
        """
    )
    print("Query executed successfully")
except Exception as e:
    print(f"Oops: {e}")
    
for row in rows:
    print(row.artist, row.song, row.length)

Query executed successfully
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Table 2 Description: The table has artist, song, first_name, last_name, sessionId, userId, and itemInSession columns. Primary key is ((SessionId, userId), itemInSession) 
# where (SessionId, usreId) represents partition key and itemInSession represents clustering cloumn which is sorted in ascending order as we need to run query2
# for particular values of sessionId and userId with songs being sorted by itemInSession column. 

try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS session_user_item_table
        (artist text, song text, first_name text, last_name text, sessionId int, userId int, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)
        """
    )
    print("Table 2 created successfully")
except Exception as e:
    print(f"Oops: {e}")
                    

Table 2 created successfully


In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_user_item_table (artist, song, first_name, last_name, sessionId, userId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[8]), int(line[10]), int(line[3])))
        
print("Table 2 filled successfully")

Table 2 filled successfully


In [15]:
try: 
    rows = session.execute(
        """
        SELECT artist, song, first_name, last_name FROM session_user_item_table WHERE sessionId=182 and userId = 10
        """
    )
    print("Query executed successfully")
except Exception as e:
    print(f"Oops: {e}")
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Query executed successfully
Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
## Table 3 Description: The table has first_name, last_name, song, and userId columns. Primary key is ((song), userId) where song is partition key and userId is clustering column
# as we need to run query3 for particular value of song, moreover userId is needed to generate unique primary keys.

try:
    session.execute(
        """
        CREATE TABLE IF NOT EXISTS song_user_table
        (first_name text, last_name text, song text, userId int, PRIMARY KEY ((song), userId))
        """
    )
    print("Table 3 created successfully")
except Exception as e:
    print(f"Oops: {e}")

                    

Table 3 created successfully


In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_table (first_name, last_name, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))
        
print("Table 3 filled successfully")

Table 3 filled successfully


In [20]:
try: 
    rows = session.execute(
        """
        SELECT first_name, last_name FROM song_user_table WHERE song = 'All Hands Against His Own'
        """
    )
    print("Query executed successfully")
except Exception as e:
    print(f"Oops: {e}")
    
for row in rows:
    print(row.first_name, row.last_name)

Query executed successfully
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
try:
    session.execute(
        """
        DROP TABLE sparkify.session_item_table
        """
    )
    session.execute(
        """
        DROP TABLE sparkify.session_user_item_table
        """
    )
    session.execute(
        """
        DROP TABLE sparkify.song_user_table
        """
    )
    print("Tables droped successfully")
except Exception as e:
    print(f"Oops: {e}")

Tables droped successfully


### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()